In [15]:
import numpy as np
import pandas as pd
from boto3 import client
from pymongo import MongoClient
from datetime import datetime, timedelta
from sqlalchemy.orm.collections import collection

In [16]:
def pulling_data_db():
    
    client = MongoClient('localhost', 27017)
    db = client['financeStockData']
    collection = db['stockData']
    table = []
    
    query1 = {'Date': datetime(2024, 1, 2)}
    dt = collection.find(query1)
    
    for data in dt:
        table.append(data)
 
    fin_data = pd.DataFrame(table)
    return fin_data

In [17]:
df = pulling_data_db()
df

""


In [ ]:
query1 = {'Date': '2024-01-02'}